In [118]:
%run func_def.ipynb

imports loaded.

Pre-Processing func:
********************
    get_height(item)
    get_weight(item)
    get_reach(item)
    get_std_reach(item, height, reach_list, height_list)
    get_age(item)
    get_std_age(item, age_list)
    get_win_perc(name, date, df)
    get_lose_streak_data_frame(name, date, df)
    get_win_streak_data_frame(name, date, df)
    get_streak_data_frame(name, date, df)
    get_win_streak(name, date, df)
    get_lose_streak(name, date, df)
    get_fighter_stats(name, data )
    add_columns(data, column_list)

Feature Engineering func:
*************************
    get_stat(index, data, stat='height', fo='')
    get_swaped_row(index, data)
    get_wins(combo, data)
    get_wins_ratio(combo, data)

stats_model_training func:
********************************
    get_sig_features(data, sig_r)
    get_r_score(X, y)
    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)
    set_random_forest_reg(X_columns, y_columns, data)
    set_predictions(mod

### <span style='color:blue'> Insert new fight: </span> 

In [119]:
fighter = 'Islam Makhachev'
opponent = 'Alexander Volkanovski'
print(fighter + ' vs ' + opponent)

Islam Makhachev vs Alexander Volkanovski


### Fight Preview(load statistics data frame):

In [120]:
loc = '../data/'
file = 'ufc_fighters_stats_processed.csv'
stats = pd.read_csv(loc+file)
stats = stats[stats.columns[1:]]
get_fighters_stats(fighter=fighter, opponent=opponent, data=stats)

,Islam Makhachev,Alexander Volkanovski
wins,23,25
losses,1.0,1.0
draws,0,0
height,177.8,167.64
weight,69.75,65.25
reach,177.8,180.34
stance,Southpaw,Orthodox
age,31.0,34.0
slpm,2.37,6.79
str. acc.,0.59,0.57


### load main data frame:

In [121]:
path = '../data/fight_stat_data_trained.csv'
df = pd.read_csv(path)
df = df[df.columns[0:len(df.columns)-4]]
df.date = pd.to_datetime(df.date)
print('Data Frame loaded: '+str(df.shape))
df.tail(3)

Data Frame loaded: (6731, 22)


,set,date,fighter,opponent,result,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
6728,test,2017-07-29,Aleksandra Albu,Kailin Curran,win,U-DEC:,0.011068,-0.133163,1.590190,0.001133,0.157318,-0.236546,0.544717,-0.194404,0.584680,0.331559,0.477663,-1.248515,0.270739,2.060202,0.599149,-2.164903
6729,train,2017-07-29,Jarred Brooks,Eric Shelton,win,S-DEC:,0.011068,-0.133163,1.590190,0.001133,0.960129,0.218076,0.779264,0.377632,-0.294627,0.331559,-0.151239,-0.070965,-0.507334,0.015847,0.026948,-1.087573
6730,validation,2017-07-29,Drew Dober,Joshua Burkman,win,KO/TKO:Punch,1.365610,0.167994,0.011143,0.001133,-0.985573,-0.656198,-0.594516,-0.194404,1.116061,-0.236000,0.611992,0.013146,0.985268,0.143619,0.026948,-2.164903


In [122]:
match = set_new_fight(fighter=fighter, opponent=opponent, data=df)
match

,set,method,date,result,fighter,opponent,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,None,None,2023-02-23,None,Islam Makhachev,Alexander Volkanovski,None,None,None,None,1.71,0.29,0.15,1.1,-4.42,0.02,-2.58,0.06,-0.004158,0,-1,0


### Load fight statistics prediction model:

In [123]:
stat_pred_reg = load_trained_model(filename_path='../global_variables/stats_pred_random_forest_reg.pkl')
X = stat_pred_reg.feature_names_in_
y = ['kd_dif','str_dif','td_dif','sub_dif']
feature_importance = stat_pred_reg.feature_importances_

fi = pd.DataFrame(data=feature_importance, index=X, columns=['feature_importances']).sort_values(by=['feature_importances'], ascending=False)
fi

../global_variables/stats_pred_random_forest_reg.pkl loaded.


,feature_importances
td avg._dif,0.205326
slpm_dif,0.154638
sapm_dif,0.129441
sub. avg._dif,0.122064
td def._dif,0.066731
str. acc._dif,0.062140
td acc._dif,0.061202
str. def_dif,0.057431
win_perc_dif,0.049351
win_strk_dif,0.046791


#### Load predictions to match:

In [124]:
stats_pred_list = stat_pred_reg.predict(match[stat_pred_reg.feature_names_in_])

i = 0
for stat in ['kd_dif','str_dif','td_dif','sub_dif']:
    match.loc[0,stat] = stats_pred_list[0][i]
    i += 1

match

,set,method,date,result,fighter,opponent,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,None,None,2023-02-23,None,Islam Makhachev,Alexander Volkanovski,-0.50353,-0.208166,0.92578,0.336553,1.71,0.29,0.15,1.1,-4.42,0.02,-2.58,0.06,-0.004158,0,-1,0


### <span style='color:green'> Statistics prediction to the match: </span>

In [125]:
match[y]

,kd_dif,str_dif,td_dif,sub_dif
0,-0.50353,-0.208166,0.92578,0.336553


### Load fight result prediction model:

In [126]:
result_pred_clf = load_trained_model(filename_path='../global_variables/result_pred_random_forest_clf.pkl')
X = result_pred_clf.feature_names_in_
y = 'result'
feature_importance = result_pred_clf.feature_importances_

fi = pd.DataFrame(data=feature_importance, index=X, columns=['feature_importances']).sort_values(by=['feature_importances'], ascending=False)
fi

../global_variables/result_pred_random_forest_clf.pkl loaded.


,feature_importances
str_dif,0.345877
kd_dif,0.118405
td_dif,0.087482
sub_dif,0.079351
slpm_dif,0.052311
td def._dif,0.043547
td avg._dif,0.040059
sapm_dif,0.038678
str. def_dif,0.033933
td acc._dif,0.032574


In [127]:
result_pred_list = result_pred_clf.predict(match[result_pred_clf.feature_names_in_])
match.loc[0,y] = result_pred_list[0]
winner = get_winner(match=match) #saving winner before 'result' becoming a feature
match

,set,method,date,result,fighter,opponent,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,None,None,2023-02-23,lose,Islam Makhachev,Alexander Volkanovski,-0.50353,-0.208166,0.92578,0.336553,1.71,0.29,0.15,1.1,-4.42,0.02,-2.58,0.06,-0.004158,0,-1,0


### <span style='color:green'> Result prediction to the match(fighter win/lose): </span>

In [128]:
print(match.loc[0,y])

lose


### Load method of winning prediction model:

In [129]:
method_pred_clf = load_trained_model(filename_path='../global_variables/method_pred_random_forest_clf.pkl')
X = method_pred_clf.feature_names_in_
y = 'method'
feature_importance = method_pred_clf.feature_importances_

fi = pd.DataFrame(data=feature_importance, index=X, columns=['feature_importances']).sort_values(by=['feature_importances'], ascending=False)
fi

../global_variables/method_pred_random_forest_clf.pkl loaded.


,feature_importances
sub_dif,0.231404
kd_dif,0.151196
str_dif,0.120042
td_dif,0.095627
sub. avg._dif,0.052181
td avg._dif,0.051822
sapm_dif,0.048005
slpm_dif,0.046540
td def._dif,0.045594
str. def_dif,0.043776


#### converting 'result' to feature:

In [132]:
result = match.loc[0,'result']
match.loc[0,'result'] = len(df[(df.result==result)&(df.fighter==fighter)])/len(df[df.fighter==fighter])
match

,set,method,date,result,fighter,opponent,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,None,None,2023-02-23,0.0,Islam Makhachev,Alexander Volkanovski,-0.50353,-0.208166,0.92578,0.336553,1.71,0.29,0.15,1.1,-4.42,0.02,-2.58,0.06,-0.004158,0,-1,0


In [143]:
method_pred_list = method_pred_clf.predict(match[method_pred_clf.feature_names_in_])
match.loc[0,y] = method_pred_list[0]
method = match.loc[0,'method']
match

,set,method,date,result,fighter,opponent,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,None,decision,2023-02-23,0.0,Islam Makhachev,Alexander Volkanovski,-0.50353,-0.208166,0.92578,0.336553,1.71,0.29,0.15,1.1,-4.42,0.02,-2.58,0.06,-0.004158,0,-1,0


In [144]:
gp = pd.read_csv('../global_variables/ml_models_parameters.csv')

result_accuracy = -1
method_accuracy = -1
for model in gp.ml_model:
    if 'result_' in model:
        result_accuracy = gp.loc[gp.ml_model == model,'accuracy']
    if 'method_' in model:
        method_accuracy = gp.loc[gp.ml_model == model,'accuracy']    

### <span style='color:green'> Match predictions summary: </span>

In [149]:
print('--------------------------------------')
print('%s vs %s:' %(fighter, opponent))
print('--------------------------------------')
print('Winner: %s [Accuracy = %.3f%%]' %(winner,100*result_accuracy))
print('By: %s [Accuracy = %.3f%%]' %(method,100*method_accuracy))
print('--------------------------------------')
get_fighters_stats(fighter=fighter, opponent=opponent, data=stats)

--------------------------------------
Islam Makhachev vs Alexander Volkanovski:
--------------------------------------
Winner: Alexander Volkanovski [Accuracy = 86.701%]
By: decision [Accuracy = 72.235%]
--------------------------------------


,Islam Makhachev,Alexander Volkanovski
wins,23,25
losses,1.0,1.0
draws,0,0
height,177.8,167.64
weight,69.75,65.25
reach,177.8,180.34
stance,Southpaw,Orthodox
age,31.0,34.0
slpm,2.37,6.79
str. acc.,0.59,0.57
